In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [2]:
from datasets import load_dataset

dataset = load_dataset("fashion_mnist")

Found cached dataset fashion_mnist (C:/Users/Asus VivoBook/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/8d6c32399aa01613d96e2cbc9b13638f359ef62bb33612b077b4c247f6ef99c1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [7]:
dataset["train"][0]["image"].size

(28, 28)

In [52]:
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16 * 7 *7, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [53]:
model = CNN()

In [54]:
x = torch.randn(64, 1, 28, 28)

In [55]:
model(x)

tensor([[ 3.6210e-01,  6.1276e-02,  3.9492e-01,  5.4865e-02, -2.4622e-01,
         -4.5538e-02, -3.2923e-01, -8.6868e-02, -2.7230e-01,  2.6763e-01],
        [ 3.9795e-01,  3.8829e-02,  4.2814e-01, -1.0295e-01, -3.3889e-01,
         -1.0748e-01, -5.2552e-01,  9.2619e-02, -2.9414e-01,  3.3144e-01],
        [ 2.2639e-01, -1.6436e-02,  3.6940e-01,  5.6007e-03, -2.5027e-01,
         -1.1984e-01, -5.5807e-01,  8.7116e-03, -2.1144e-01,  3.7039e-01],
        [ 3.0275e-01,  3.3400e-02,  5.7700e-01,  1.2718e-02, -2.9255e-01,
         -1.1348e-01, -2.9942e-01, -1.4505e-02, -1.6089e-01,  3.7282e-01],
        [ 2.3811e-01,  4.1060e-02,  4.2610e-01,  1.2021e-01, -2.8817e-01,
         -2.3772e-01, -5.2977e-01, -8.2805e-02, -1.8354e-01,  1.9569e-01],
        [ 3.1837e-01, -1.2511e-01,  3.4871e-01, -1.7195e-01, -3.0574e-01,
         -1.2311e-01, -4.3229e-01,  3.7181e-02, -2.7130e-01,  3.3030e-01],
        [ 3.2365e-01,  3.6572e-02,  3.8569e-01,  4.3064e-02, -2.3169e-01,
         -1.5585e-01, -4.0006e-0

In [56]:
device = torch.device("cpu")

In [57]:
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [58]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

In [59]:
training_data[0][0].shape

torch.Size([1, 28, 28])

In [60]:
train_loader = DataLoader(dataset = training_data, batch_size= batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_data, batch_size=batch_size, shuffle=True)

In [61]:
model = CNN().to(device)

In [62]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [63]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    print(f"Epoch: {epoch}:Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
    print(f"Epoch: {epoch}:Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Epoch: 0:Accuracy on training set: 84.93
Epoch: 0:Accuracy on test set: 83.57
Epoch: 1:Accuracy on training set: 87.23
Epoch: 1:Accuracy on test set: 86.65
Epoch: 2:Accuracy on training set: 88.59
Epoch: 2:Accuracy on test set: 87.28
Epoch: 3:Accuracy on training set: 89.10
Epoch: 3:Accuracy on test set: 88.05
Epoch: 4:Accuracy on training set: 89.38
Epoch: 4:Accuracy on test set: 88.31
Epoch: 5:Accuracy on training set: 90.19
Epoch: 5:Accuracy on test set: 88.68
Epoch: 6:Accuracy on training set: 89.62
Epoch: 6:Accuracy on test set: 88.45
Epoch: 7:Accuracy on training set: 90.54
Epoch: 7:Accuracy on test set: 88.93
Epoch: 8:Accuracy on training set: 90.50
Epoch: 8:Accuracy on test set: 89.17
Epoch: 9:Accuracy on training set: 90.99
Epoch: 9:Accuracy on test set: 89.15


In [65]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 90.99
Accuracy on test set: 89.15


In [66]:
torch.save(model, "cnn_model.pt")

In [69]:
index_class_mapper = {
    "0" : "tricou",
    "1" : "pantaloni",
    "2" : "pulover",
    "3" : "rochie",
    "4" : "palton",
    "5" : "sandale",
    "6" : "cămașă",
    "7" : "adidași",
    "8" : "geantă",
    "9" : "botine"
}

In [80]:
import cv2
from PIL import Image

In [117]:
img = cv2.imread("camasa.jpg")

In [118]:
img = cv2.resize(img, (28, 28))

In [119]:
img = 255 -  cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [120]:
pil_img = Image.fromarray(img)

In [121]:
pil_img

In [122]:
tens = transforms.ToTensor()(pil_img)

In [123]:
tens.shape

torch.Size([1, 28, 28])

In [124]:
tens.squeeze()

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [89]:
dir(tens)

['H',
 'T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed_

In [97]:
model.eval()

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)

In [125]:
torch.sort(model(tens.unsqueeze(dim=1))[0], descending = True).indices.tolist()

[6, 8, 4, 0, 2, 3, 5, 7, 1, 9]

In [110]:
training_data[0]

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0039, 0.0000, 0.0000, 0.0510,
           0.2863, 0.0000, 0.0000, 0.0039, 

In [ ]:
index_class_mapper = {
    "0" : "tricou",
    "1" : "pantaloni",
    "2" : "pulover",
    "3" : "rochie",
    "4" : "palton",
    "5" : "sandale",
    "6" : "cămașă",
    "7" : "adidași",
    "8" : "geantă",
    "9" : "botine"
}

In [133]:
acceptable = {
    "top" : [0, 2, 4, 6],
    "bottom" : [1, 3],
    "foot" : [5, 7, 9]
}

In [134]:
def predict_img(img, body_part):
    img = cv2.resize(img, (28, 28))
    img = 255 - cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    pil_img = Image.fromarray(img)
    tens = transforms.ToTensor()(pil_img)
    indices = torch.sort(model(tens.unsqueeze(dim=1))[0], descending = True).indices.tolist()
    acceptable_clothes = acceptable[body_part]
    for i in indices:
        if i in acceptable_clothes:
            predicted_class = index_class_mapper[str(i)]
            return predicted_class
    return None

In [137]:
predict_img(cv2.imread("camasa.jpg"), "top")

'cămașă'